In [1]:
import numpy as np
import os
import cv2
from tqdm import tqdm


%matplotlib inline


In [18]:
# configure path according to your dataset

MEN_DATA_PATH = '/home/nikhil-saxena/Downloads/Question_1/dataset/men'
WOMEN_DATA_PATH = '/home/nikhil-saxena/Downloads/Question_1/dataset/women'
#DATA_PATH = '/home/nikhil-saxena/Downloads/Question_1/dataset/f'

In [19]:
# You can cheange as you want 
# this will we your eigen faces size
target_dim = (120,120)



# if your memory is not enough then you can reduce the number of images
# -1 indicate all images and 1000 indicate 1000 images
SPLIT_SIZE = 10000


In [20]:
image_width= 0
image_height= 0

In [21]:
def is_image_file(filename):
    extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    return filename.lower().endswith(extensions)

In [33]:
def list_images_by_class(base_dir):

    data = {}
    
    # Loop through each item in the base directory
    for class_name in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_name)
    
        # Check if it's a directory
        if os.path.isdir(class_path):
            images = []
    
            for file in os.listdir(class_path):
                if is_image_file(file):
                    images.append(os.path.join(class_path, file))
    
            data[class_name] = images
    
    return data

In [35]:
#men = list_images_by_class(MEN_DATA_PATH)
#women = list_images_by_class(WOMEN_DATA_PATH)
men = list_images_by_class(DATA_PATH)

In [37]:
import random
import cv2
from tqdm import tqdm

# Combine all images from both men and women dictionaries into one list
all_images_paths = [img for key, value in men.items() for img in value]

# Randomly sample 500 images from the combined list
sample_images = all_images_paths

all_image = []
for image in tqdm(sample_images, total=len(sample_images)):
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_dim)
    all_image.append(img)


100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 3168.52it/s]


In [38]:
all_image = np.array(all_image)
all_image.shape

(2000, 120, 120)

In [39]:
all_image = all_image[:SPLIT_SIZE]  
all_image.shape

(2000, 120, 120)

In [40]:
# it give all the eigen faces in sorted order (sorting critria is eigen values)
def principalComponentAnalysis(X):

    total_images = X.shape[0]
    image_height, image_width = X.shape[1], X.shape[2]
    
    flattened_images = X.reshape(total_images, -1)
    
    mean_face = np.mean(flattened_images, axis=0)
    centered_images = flattened_images - mean_face

    # Compute covariance matrix
    covariance_matrix = np.dot(centered_images, centered_images.T) / total_images
    
    # Eigen decomposition
    eigen_values, eigen_vectors_temp = np.linalg.eig(covariance_matrix)
    
    # Compute actual eigenfaces
    eigen_vectors = np.dot(centered_images.T, eigen_vectors_temp)
    eigen_vectors = eigen_vectors / np.linalg.norm(eigen_vectors, axis=0)

    # Sort eigenvectors by eigenvalues
    sorted_indices = np.argsort(eigen_values)[::-1]
    eigen_values = eigen_values[sorted_indices]
    eigen_vectors = eigen_vectors[:, sorted_indices]

    # Select top `numComponents` eigenvectors
    # eigen_vectors = eigen_vectors[:, :]
    preserved_variance = np.sum(eigen_values[:]) / np.sum(eigen_values)
    
    return eigen_values, eigen_vectors, preserved_variance, mean_face


In [41]:
eigen_values , eigen_faces ,_, mean_face = principalComponentAnalysis(all_image)
eigen_faces.shape

(14400, 2000)

In [42]:
np.save('eigen_faces_f.npy', eigen_faces)
np.save('mean_faces_f.npy', mean_face)